# Finding duplicates and near duplicates in Speakleash - the embedding approach

The task is to find duplicates nad near duplicates of documents is Speakleash. We should flag texts generated using a template and automatically translated texts as near duplicates. 

The requirements:

We need to be able to efficiently compare a lot of documents.
We need to be able to run the search multiple times (for each new batch of documents).
We need to be able to continue the work on different machines, so it should be possible to transfer the results of the work we've done locally.
The method we used for documents will need to be applicable for parts of documents as well.

The idea:

To identify documents that are not only exact duplicates but also slightly modified copies, we need a good representation of each document. We will use embeddings.

We will use a vector database to store the embeddings. This will allow us to save the work, transfer it to another machine and continue there. Eventually, we might use a database that is deployed on a server. 


In [64]:
import chromadb
from tqdm import tqdm
import os
from speakleash import Speakleash

## get data from speakleash

In [5]:
replicate_to = "/Users/maciej/Desktop/spichlerz/datasets"

sl = Speakleash(replicate_to)

In [148]:
# get texts with metadata
test_data_meta = sl.get("forum_parenting_pl_corpus").ext_data

## initialize the database

In [37]:
client = chromadb.PersistentClient(path="/Users/maciej/Desktop/spichlerz/chroma")

In [39]:
# make a new collection. For now we will use the default embedding function
collection = client.create_collection("speakleash")

## create embeddings for all the documents in the dataset and save to the database

In [100]:
def process_documents(dataset):
    for doc in tqdm(dataset, desc="Embedding documents"):
        id = doc[1].get("url")
        collection.add(documents=doc[0],
            ids=id)

In [95]:
process_documents(test_data_meta)

Embedding documents: 30338it [1:51:09,  4.55it/s]


Processing 30338 documents took 1:51:09. The database weighs 54,5 MB. Estimates for 50 000 000 documents: 81,3 days; 89,82GB. Space requirement doesn't look great. Time is definitely an issue to solve.

### query the database to find similar documents

In [ ]:
# this is a part of the first document
test_text = """Witam na forum, jestem Mamą dwójki dzieci: przechodzącej okres buntu dwulatka Córeczki i dwumiesięcznego Synka. Mieszkam w W-wie, na co dzień zajmuję się dziećmi i niektórzy twierdzą (nie bez przyczyny) że internet moim drugim domem :) To tyle słowem wstępu, reszta na bieżąco.
I my dołączamy :)
I ja, podobnie, jak Ania, jestem mamą Dwójki, nam tez bunt dwulatka nie jest obcy ;) 
Obecnie jestem na urlopie macierzyńskim, ale w lutym zamierzam wrócić na pół etatu do szkoły, w której uczę.
Madziu witaj
jakie piekne bąble na zdjeciach ;)
czego uczysz w szkole?
Witaj Aniu,Madziu i Małgosiu :)
mnie tez jest znany temat początku buntu dwulatka :p
Madzilek ( Skabza) rany ile ja Cie nie widziałam- ostatnio chyba jak Maja uczyła się chodzic ;)mam nadzieje, ze dowiem sie na tym forum co u WAS słychacwy mieszkacie wciaz w tym samym miejscu czy juz sie przeprawadziliscie?
Witajcie
Jestem mamą 2-letniego jedynaka :) 
Z Anią i Magdą znamy się z innych miejsc netowych :)
Fajnie, że powstało to nowe forum, zawsze to jakaś alternatywa, nowe pomysły i nowe możliwości. Zarejestrowałam się z ciekawości i pewnie częściej będę czytać co u Was i Waszych pociech niż pisać. W każdym razie pozdrawiam Was serdecznie i proszę o ucałowanie ode mnie dzieciaczków.
- Nenyah
Nenyah- ucałuj i swojego jedynaka ;)mamy nadzieje, że zadomowimy się tutaj  i będzie naprawdę fajniemiłej nocy
Witaj Nenyah :) 
Małgosiu bij zabij,ale ja Cię nie kojarze,nawet na zdjecie patrze i wstyd mi :rolleyes: :)
Madzia bo ja sie potem mało udzielałam na N jako malgosiap  ;)"""

In [96]:
test_result = collection.query(
    query_texts=[test_text],
    n_results=10
)

In [97]:
test_result

{'ids': [['test',
   'https://forum.parenting.pl/topic/23-witam-serdecznie/',
   'https://forum.parenting.pl/topic/2272778-hello/',
   'https://forum.parenting.pl/topic/847279-dzien-dobry/',
   'https://forum.parenting.pl/topic/263172-witam-serdecznie/',
   'https://forum.parenting.pl/topic/143994-dla-wszystkich-mam-w-dniu-swieta/',
   'https://forum.parenting.pl/topic/636153-witam-serdecznie/',
   'https://forum.parenting.pl/topic/2951354-czesc-jestem-dorota/',
   'https://forum.parenting.pl/topic/320783-dobry-wieczor/',
   'https://forum.parenting.pl/topic/2545358-hej/']],
 'distances': [[0.0,
   0.0,
   0.2716042399406433,
   0.27208831906318665,
   0.2748403251171112,
   0.2756606936454773,
   0.27697890996932983,
   0.2783462405204773,
   0.2789933681488037,
   0.2798767685890198]],
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'embeddings': None,
 'documents': [['Witam na forum, jestem Mamą dwójki dzieci: przechodzącej okres buntu dwulatka Córeczk

As we can see, we very quickly found the exact duplicate as well as the document from which we extracted test_text.
(The exact duplicate - a part of the first document from the dataset was added to the database in a previous iteration of experiments. I didn't want to delete and create the collection from scratch so I kept it). 

Let's try querying with a slightly modified version of the test document (first post)

In [113]:
test_text2 = """Witam na forum, jestem matką dwójki dzieciaczków: przechodzącej okres rebelii dwulatka córki i syna. Mieszkam w Katowicach, na co dzień zajmuję się dziećmi i niektórzy twierdzą (nie bez przyczyny) że sieć jest moim drugim domem :) To tyle słowem wprowadzenia, reszta na bieżąco.
I my dołączamy :)
I ja, podobnie, jak Ania, jestem mamą Dwójki, nam tez bunt dwulatka nie jest obcy ;) 
Obecnie jestem na urlopie macierzyńskim, ale w lutym zamierzam wrócić na pół etatu do szkoły, w której uczę.
Madziu witaj
jakie piekne bąble na zdjeciach ;)
czego uczysz w szkole?
Witaj Aniu,Madziu i Małgosiu :)
mnie tez jest znany temat początku buntu dwulatka :p
Madzilek ( Skabza) rany ile ja Cie nie widziałam- ostatnio chyba jak Maja uczyła się chodzic ;)mam nadzieje, ze dowiem sie na tym forum co u WAS słychacwy mieszkacie wciaz w tym samym miejscu czy juz sie przeprawadziliscie?
Witajcie
Jestem mamą 2-letniego jedynaka :) 
Z Anią i Magdą znamy się z innych miejsc netowych :)
Fajnie, że powstało to nowe forum, zawsze to jakaś alternatywa, nowe pomysły i nowe możliwości. Zarejestrowałam się z ciekawości i pewnie częściej będę czytać co u Was i Waszych pociech niż pisać. W każdym razie pozdrawiam Was serdecznie i proszę o ucałowanie ode mnie dzieciaczków.
- Nenyah
Nenyah- ucałuj i swojego jedynaka ;)mamy nadzieje, że zadomowimy się tutaj  i będzie naprawdę fajniemiłej nocy
Witaj Nenyah :) 
Małgosiu bij zabij,ale ja Cię nie kojarze,nawet na zdjecie patrze i wstyd mi :rolleyes: :)
Madzia bo ja sie potem mało udzielałam na N jako malgosiap  ;)"""

In [114]:
test_result2 = collection.query(
    query_texts=[test_text2],
    n_results=10
)

In [115]:
test_result2

{'ids': [['test',
   'https://forum.parenting.pl/topic/23-witam-serdecznie/',
   'https://forum.parenting.pl/topic/3187775-nasz-drugi-cudek/',
   'https://forum.parenting.pl/topic/2951354-czesc-jestem-dorota/',
   'https://forum.parenting.pl/topic/320783-dobry-wieczor/',
   'https://forum.parenting.pl/topic/263172-witam-serdecznie/',
   'https://forum.parenting.pl/topic/2042642-witam-drodzy-forumowicze/',
   'https://forum.parenting.pl/topic/10884-dzieciaczki-rocznik-2003/',
   'https://forum.parenting.pl/topic/136820-rybka-z-narybkiem/',
   'https://forum.parenting.pl/topic/1597996-witajcie/']],
 'distances': [[0.06205762177705765,
   0.06205762177705765,
   0.2444087564945221,
   0.2544991374015808,
   0.26348358392715454,
   0.2801579535007477,
   0.2803778350353241,
   0.2810214161872864,
   0.28766536712646484,
   0.28899550437927246]],
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'embeddings': None,
 'documents': [['Witam na forum, jestem Mamą dw

The first two documents that the query returned are correctly identified as very similar. The difference in the distance between the true positives and the rest of the returned documents (which are semantically related threads) seems significant enough to set a reasonable threshold.

Let's try something harder. We'll google translate the a selected post from test_text to English and then back to Polish and query the database with the result.

In [119]:
test_text3 = """witaj madziu
jakie piękne bąbelki na zdjęciach ;)
czego uczysz w szkole?"""

In [120]:
test_result3 = collection.query(
    query_texts=[test_text3],
    n_results=10
)

In [121]:
test_result3

{'ids': [['https://forum.parenting.pl/topic/519543-witam-wszystkie-mamy-ktorych-dzieci-koncza-roczek/',
   'https://forum.parenting.pl/topic/1038311-hejka/',
   'https://forum.parenting.pl/topic/969931-witam/',
   'https://forum.parenting.pl/topic/4093915-konkurs-iwoniczanki/',
   'https://forum.parenting.pl/topic/931869-witam-d/',
   'https://forum.parenting.pl/topic/760262-witam-wszystkich/',
   'https://forum.parenting.pl/topic/1244280-witamy/',
   'https://forum.parenting.pl/topic/962343--/',
   'https://forum.parenting.pl/topic/968344-witam-wieczornie/',
   'https://forum.parenting.pl/topic/1532010-witam-wszystkich/']],
 'distances': [[0.47984904050827026,
   0.48951154947280884,
   0.49130502343177795,
   0.49320051074028015,
   0.49938178062438965,
   0.5119808912277222,
   0.5146253108978271,
   0.5181032419204712,
   0.5182576775550842,
   0.5194111466407776]],
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'embeddings': None,
 'documents': [['s

Not great, we did not find the right documents. But the results have large distances, so at least we didn't get false negatives.

Let's try the same thing but this time we'll translate the entire document

In [122]:
test_text4 = """Witam na forum, jestem mamą dwójki dzieci: dwuletniej córki i syna przechodzącego okres buntu. Mieszkam w Katowicach, na co dzień zajmuję się dziećmi i niektórzy (nie bez powodu) mówią, że internet to mój drugi dom :) To tyle tytułem wstępu, reszta na bieżąco.
ja mój dołączamy :)
ja, podobnie, jak Ania, jestem mamą Dwójki, nam tez bunt dwulatka nie jest obcy ;)
Obecnie jestem na urlopie macierzyńskim, ale w planach powrót na pół etatu do szkoły, w której uczę.
Madziu witaj
jakie piekne bąble na zdjeciach ;)
czego uczysz w szkole?
Witaj Aniu, Madziu i Małgosiu :)
mnie tez jest dziwny temat poczatku buntu dwulatka :p
Madzilek (Skabza) rany ile ja Cie nie widzial- ostatnio chyba jak Maja pomysla sie chodzic;)mam wspolna, ze dowiem sie na tym forum co u WAS slyacwy mieszkacie wciaz w tym samym miejscu czy juz sie przeprawadziliscie?
Witajcie
Jestem mamą 2-letnią jedynąka :)
Z Anią i Magdą znamy się z innych miejsc netowych :)
Fajnie, że wygląd na nowe forum, zawsze na jakąś alternatywę, nowe pomysły i nowe możliwości. Zarejestrowałem się z tego i pewnie będę czytać co u Was i pomóc pomagać niż pisać. W każdym razie pozdrawiam Was serdecznie i proszę o ucałowanie ode mnie dzieciaczków.
- Nenyah
Nenyah- ucałuj i jej jedynaka ;)mamy to, że zadomowimy się tutaj i będzie naprawdę fajniemiłej nocy
Witaj Nenyah :)
Małgosiu bij zabij,ale ja Cię nie kojarze,nawet na zdjecie patrze i wstyd mi :rolleyes: :)
Madzia bo ja sie potem pomoglam na N jako malgosiap ;) """

In [123]:
test_result4 = collection.query(
    query_texts=[test_text4],
    n_results=10
)

In [124]:
test_result4

{'ids': [['test',
   'https://forum.parenting.pl/topic/23-witam-serdecznie/',
   'https://forum.parenting.pl/topic/2951354-czesc-jestem-dorota/',
   'https://forum.parenting.pl/topic/320783-dobry-wieczor/',
   'https://forum.parenting.pl/topic/263172-witam-serdecznie/',
   'https://forum.parenting.pl/topic/1597996-witajcie/',
   'https://forum.parenting.pl/topic/2042642-witam-drodzy-forumowicze/',
   'https://forum.parenting.pl/topic/136820-rybka-z-narybkiem/',
   'https://forum.parenting.pl/topic/3187775-nasz-drugi-cudek/',
   'https://forum.parenting.pl/topic/2272778-hello/']],
 'distances': [[0.07189219444990158,
   0.07189219444990158,
   0.2710663676261902,
   0.28770047426223755,
   0.29061076045036316,
   0.2930251359939575,
   0.29346445202827454,
   0.29367008805274963,
   0.3009015917778015,
   0.30230382084846497]],
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'embeddings': None,
 'documents': [['Witam na forum, jestem Mamą dwójki dzieci: pr

Bullseye! 

One last thing: do we have duplicates/near duplicates in this particular datasets?

In [131]:
test_result4

{'ids': [['test',
   'https://forum.parenting.pl/topic/23-witam-serdecznie/',
   'https://forum.parenting.pl/topic/2951354-czesc-jestem-dorota/',
   'https://forum.parenting.pl/topic/320783-dobry-wieczor/',
   'https://forum.parenting.pl/topic/263172-witam-serdecznie/',
   'https://forum.parenting.pl/topic/1597996-witajcie/',
   'https://forum.parenting.pl/topic/2042642-witam-drodzy-forumowicze/',
   'https://forum.parenting.pl/topic/136820-rybka-z-narybkiem/',
   'https://forum.parenting.pl/topic/3187775-nasz-drugi-cudek/',
   'https://forum.parenting.pl/topic/2272778-hello/']],
 'distances': [[0.07189219444990158,
   0.07189219444990158,
   0.2710663676261902,
   0.28770047426223755,
   0.29061076045036316,
   0.2930251359939575,
   0.29346445202827454,
   0.29367008805274963,
   0.3009015917778015,
   0.30230382084846497]],
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'embeddings': None,
 'documents': [['Witam na forum, jestem Mamą dwójki dzieci: pr

In [129]:
test_result4['distances'][0]

[0.07189219444990158,
 0.07189219444990158,
 0.2710663676261902,
 0.28770047426223755,
 0.29061076045036316,
 0.2930251359939575,
 0.29346445202827454,
 0.29367008805274963,
 0.3009015917778015,
 0.30230382084846497]

In [146]:
def test_dataset(dataset, collection, threshold=0.1):
    for doc in dataset:
        query_result = collection.query(query_texts=[doc[0]],n_results=3)
        distances = query_result['distances'][0]
        if len(distances) > 1:
            duplicates = [(index, value) for index, value in enumerate(distances) if value <= threshold]
            if len(duplicates) > 0:
                for item in duplicates:
                    if query_result["ids"][0][item[0]] != doc[1].get("url"):
                        print(f'document: {doc[1].get("url")} has duplicates: {query_result["ids"][0][item[0]]}\t score: {item[1]} \n\n')
    print("test completed!")

In [149]:
test_dataset(test_data_meta, collection)

document: https://forum.parenting.pl/topic/23-witam-serdecznie/ has duplicates: test	 score: 0.0 


document: https://forum.parenting.pl/topic/29917-w-oczekiwaniu-na-dziecko-nowy-konkurs-fotograficzny/ has duplicates: https://forum.parenting.pl/topic/29713-w-oczekiwaniu-na-dziecko-nowy-konkurs-fotograficzny-zakonczony/	 score: 0.0 


document: https://forum.parenting.pl/topic/704316-wybor-odpowiednich-zabawek-i-zajec-w-zaleznosci-od-wieku-dziecka/ has duplicates: https://forum.parenting.pl/topic/704317-wybor-odpowiednich-zabawek-i-zajec-w-zaleznosci-od-wieku-dziecka/	 score: 0.007181244436651468 


document: https://forum.parenting.pl/topic/704317-wybor-odpowiednich-zabawek-i-zajec-w-zaleznosci-od-wieku-dziecka/ has duplicates: https://forum.parenting.pl/topic/704316-wybor-odpowiednich-zabawek-i-zajec-w-zaleznosci-od-wieku-dziecka/	 score: 0.007181244436651468 


document: https://forum.parenting.pl/topic/752295-migotka27-dzis-swietuje/ has duplicates: https://forum.parenting.pl/topic/

We found a lot - both exact and near duplicates. They can be examined manually.
It seems that the 'near duplicates' in most of the cases are the same posts in different threads (usually copies with no replies)

### To do: experiment with parallelization for faster processing

In [99]:
from joblib import Parallel, delayed

In [102]:
client.create_collection("speakleash2")

Collection(name=speakleash2)

In [104]:
collection2 = client.get_collection("speakleash2")

In [106]:
# commiting crimes - changed the name of the collection inside of the function :)
def process_documents_parallel(dataset):
    for doc in tqdm(dataset, desc="Embedding documents"):
        id = doc[1].get("url")
        collection2.add(documents=doc[0],
            ids=id)

In [112]:
Parallel(n_jobs=8)(
    delayed(process_documents_parallel)(doc) for doc in tqdm(test_data_meta, desc="Processing documents"))



Processing documents: 0it [00:00, ?it/s]

Processing documents: 3it [00:00, 23.86it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 



Processing documents: 8it [00:00, 30.42it/s]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


PicklingError: Could not pickle the task to send it to the workers.



Processing documents: 8it [00:11, 30.42it/s]

whoops... Need to find a smarter way to do it.